<a href="https://colab.research.google.com/github/MihailMV/Various/blob/master/Conway's_Reverse_Game_of_Life_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

In [ ]:
CNT_LIFE = 20_000
CNT_STEP_LIFE = 15
HEIGHT = 25
WIDTH = 25

BATCH_SIZE = 2_000

In [ ]:
import time

class Configurations:
    def __init__(self):
        self.verbose = True
        self.verbose_step = 10
        self.cnt_epoch = 30
        self.device = 'cpu'
        self.file_log = 'log.txt'
        self.lr = 3e-3
        self.scheduler_params = dict(
                factor = 0.3
                , patience=100
                , threshold= 0.006
                , verbose=True
                )


class Сonstructor:
    def __init__(self, model, loss, metrics, optimizer, scheduler = None, configurations = None):
        self.model = model
        self.loss = loss
        self.metrics = metrics
        self.configurations = configurations
        self.optimizer = optimizer(self.model.parameters(), lr = self.configurations.lr)
        if scheduler != None:
          self.scheduler = scheduler(self.optimizer, **self.configurations.scheduler_params)
        self.epoch = 0
        self.best_score = None
        self.history = {'epoch':[], 'loss':[], 'metrics':{}}
        for metric in self.metrics.keys():
            self.history['metrics'][metric] = []
        
        self.loss = self.loss.to(self.configurations.device)
        self.model = self.model.to(self.configurations.device)

    
    def one_epoch(self, dataloader, mode = 'train'):
        if mode == 'train':
            self.model.train()
            self.history['epoch'].append(self.epoch)
        else:
            self.model.eval()        
        str_result = ''
        cnt = 0
        sum_loss = 0
        sum_metrica = {}
        for metric in self.metrics.keys():
            sum_metrica[metric] = 0
        time_start = time.time()        
        for step, (features, targets) in enumerate(dataloader):
            features = features.to(self.configurations.device, dtype = torch.float32)
            targets = targets.to(self.configurations.device, dtype = torch.float32)
            self.optimizer.zero_grad()            
            cnt += 1
            with torch.set_grad_enabled(mode == 'train'):
                predictions = self.model(features)
                predictions = predictions.to(self.configurations.device, dtype = torch.float32)

                #
                #predictions = predictions.permute(0, 2, 3, 1)
                
                #predictions = predictions.resize(predictions.shape[0] * predictions.shape[1] * predictions.shape[2],2)
                #predictions = predictions.view(-1, 1)
                #targets = targets.view(-1, 1)
                #targets = targets.long()
                #  

                loss = self.loss(predictions, targets)                
                if mode == 'train':
                    loss.backward()
                    self.optimizer.step()
                    self.scheduler.step(loss.item())                
                sum_loss += loss.item()

                for metric in self.metrics.keys():
                    sum_metrica[metric] += self.metrics[metric](predictions.data, targets)                
                str_result = f'{mode} epoch {self.epoch} '
                if step+1 != len(dataloader):
                    str_result += f'step {step+1}/{len(dataloader)}'
                str_result += f'loss = {(sum_loss/cnt):.3f} '
                str_metrics = ''
                for metric in self.metrics.keys():
                    str_metrics += f'{metric} = {(sum_metrica[metric]/cnt):.3f} '                
                
                str_result += str_metrics
                if self.configurations.verbose and step % self.configurations.verbose_step == 0:
                    print(str_result, end='\r')
                
                if step+1 == len(dataloader):
                    str_result += f'time {(time.time() - time_start):.2f}'
                    self.log(str_result)
                    self.history['loss'].append(sum_loss/cnt)
                    for metric in self.metrics.keys():
                        self.history['metrics'][metric].append(sum_metrica[metric]/cnt)        

    def fit(self, train_loader, val_loader):
        for epoch in range(self.configurations.cnt_epoch):
            for mode in ['train','val']:
                if mode == 'train':
                    dataloader = train_loader
                else:
                    dataloader = val_loader
                self.one_epoch(dataloader, mode)
            self.epoch += 1
    
    def log(self, message):
        if self.configurations.verbose:
            print(message)
        with open(self.configurations.file_log, 'a+') as logger:
            logger.write(f'{message}\n')


In [ ]:
class LifeDataset(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y
  
  def __getitem__(self,id):
    return self.x[id], self.y[id]
  
  def __len__(self):
    return len(self.x)

In [ ]:
model = torch.nn.Sequential(


     nn.Conv2d(in_channels=1, out_channels=32, kernel_size=7, padding=3)
    , nn.ReLU()
    , nn.BatchNorm2d(32)
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
    , nn.ReLU()
    , nn.BatchNorm2d(32)
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
    , nn.ReLU()
    , nn.BatchNorm2d(32)

    , nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, padding=3)
    , nn.ReLU()
    , nn.BatchNorm2d(64)
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2)
    , nn.ReLU()
    , nn.BatchNorm2d(64)
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
    , nn.ReLU()
    , nn.BatchNorm2d(64)

    #, nn.ConvTranspose2d(kernel_size=25, in_channels=128, out_channels=2)
    #, nn.ReLU()

    , nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, padding=0)
    #, nn.Softmax(dim=1)
    , nn.Sigmoid()
    )


In [ ]:
class Unet(nn.Module):
  def __init__(self):
    super(Unet, self).__init__()

    self.con11 = nn.Conv2d(kernel_size=3, in_channels=1, out_channels=8, padding=1)
    self.r11 = nn.ReLU()
    self.batch11 = nn.BatchNorm2d(8)
    self.con12 = nn.Conv2d(kernel_size=3, in_channels=8, out_channels=8, padding=1)
    self.r12 = nn.ReLU()
    self.batch12 = nn.BatchNorm2d(8)
    self.max1 = nn.MaxPool2d(kernel_size=2)

    self.con21 = nn.Conv2d(kernel_size=3, in_channels=8, out_channels=16, padding=1)
    self.r21 = nn.ReLU()
    self.batch21 = nn.BatchNorm2d(16)
    self.con22 = nn.Conv2d(kernel_size=3, in_channels=16, out_channels=16, padding=1)
    self.r22 = nn.ReLU()
    self.batch22 = nn.BatchNorm2d(16)
    self.max2 = nn.MaxPool2d(kernel_size=2)

    self.con31 = nn.Conv2d(kernel_size=3, in_channels=16, out_channels=32, padding=1)
    self.r31 = nn.ReLU()
    self.batch31 = nn.BatchNorm2d(32)
    self.con32 = nn.Conv2d(kernel_size=3, in_channels=32, out_channels=32, padding=1)
    self.r32 = nn.ReLU()
    self.batch32 = nn.BatchNorm2d(32)
    self.max3 = nn.MaxPool2d(kernel_size=2)

    self.con = nn.Conv2d(kernel_size=3, in_channels=32, out_channels=64)

    self.up41 = nn.ConvTranspose2d(kernel_size=3, in_channels=64, out_channels=32)
    self.con41 = nn.Conv2d(kernel_size=3, in_channels=64, out_channels=32, padding=1)
    self.r41 = nn.ReLU()
    self.batch41 = nn.BatchNorm2d(32)
    self.con42 = nn.Conv2d(kernel_size=3, in_channels=32, out_channels=32, padding=1)
    self.r42 = nn.ReLU()
    self.batch42 = nn.BatchNorm2d(32)

    self.up51 = nn.ConvTranspose2d(kernel_size=4, in_channels=32, out_channels=16)
    self.con51 = nn.Conv2d(kernel_size=3, in_channels=32, out_channels=16, padding=1)
    self.r51 = nn.ReLU()
    self.batch51 = nn.BatchNorm2d(16)
    self.con52 = nn.Conv2d(kernel_size=3, in_channels=16, out_channels=16, padding=1)
    self.r52 = nn.ReLU()
    self.batch52 = nn.BatchNorm2d(16)

    self.up61 = nn.ConvTranspose2d(kernel_size=2, in_channels=16, out_channels=8, stride= 2)
    self.con61 = nn.Conv2d(kernel_size=3, in_channels=16, out_channels=8, padding=1)
    self.r61 = nn.ReLU()
    self.batch61 = nn.BatchNorm2d(8)
    self.con62 = nn.Conv2d(kernel_size=3, in_channels=8, out_channels=8, padding=1)
    self.r62 = nn.ReLU()
    self.batch62 = nn.BatchNorm2d(8)

    self.up = nn.ConvTranspose2d(kernel_size=3, in_channels=8, out_channels=1, stride= 2)
    #self.r = nn.Softmax(dim=1)
    self.r = nn.Sigmoid()

  def forward(self, x):
    x1 = self.max1(self.batch12(self.r12(self.con12(self.batch11(self.r11(self.con11(x)))))))
    x2 = self.max2(self.batch22(self.r22(self.con22(self.batch21(self.r21(self.con21(x1)))))))
    x3 = self.max3(self.batch32(self.r32(self.con32(self.batch31(self.r31(self.con31(x2)))))))
    x = self.con(x3)

    x = self.up41(x)
    x = torch.cat([x,x3], dim=1)
    x = self.batch42(self.r42(self.con42(self.batch41(self.r41(self.con41(x))))))

    x = self.up51(x)
    x = torch.cat([x,x2], dim=1)
    x = self.batch52(self.r52(self.con52(self.batch51(self.r51(self.con51(x))))))

    x = self.up61(x)
    x = torch.cat([x,x1], dim=1)
    x = self.batch62(self.r62(self.con62(self.batch61(self.r61(self.con61(x))))))
    
    x = self.r(self.up(x))

    return x

In [ ]:
class LossLife(nn.Module):
  def __init__(self):
    super(LossLife, self).__init__()

    self.nn_step_life = nn.Conv2d(1,1,3,padding=(1,1))
    self.nn_step_life.weight.data = self.nn_step_life.weight.data * 0 + 1 
    self.nn_step_life.bias.data *= 0
    self.nn_step_life.requires_grad = False
  
  

  def forward(self, x, y):
    x_new = self.nn_step_life(x)
    #x_new = (x == 0) * 1. * (x_new == 3) + (x == 1) * 1. * ((x_new == 2)  + (x_new == 3))
    #x_new = x
    x = 2**x + 2**(x_new+2)
    w = (y * 9) + 1
      
    return (torch.min(torch.min((x - y * 18.).abs(), (x - y * 33.).abs()), (x - y * 34.).abs()) * w).sum() #(x_new - y).abs().sum()


In [ ]:
#model = Unet()
#model = RestNet()

#model(x_train).shape
#model(x_train[0:1]).shape
#model(x_train[0:1]).shape

#model(x_train[0:1]).shape

In [ ]:
#t = model(x_train[0:2]) #.shape, x_train[0:2].shape
#t.min()

#n = nn.ConvTranspose2d(1,1,3)
#nn.ConvTranspose2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)(x_train[0:1]).shape

#n.weight, n.bias

#y = model(x_train[0:1])

#y.argmax(dim=1) * 1.


#y = (model(x_train[0:1])> 0.5) * 1.
#y = y.view(-1) #.shape
#y.shape
#y = y.permute(0, 2, 3, 1).view(-1,2)


In [ ]:
#n(x_train[0:1])
#loss(y, y_train[0:1].view(-1))

#y.shape, y_train[0:1].view(-1).shape
#y.dtype, y_train[0:1].view(-1).dtype


In [ ]:
#loss(y, y_train[0:1].view(-1).long())

In [ ]:
# Модель для расчета одного этапа "жизни"
def one_step_life(pitch):
  nn_step_life = nn.Conv2d(1,1,3,padding=(1,1))
  nn_step_life.weight.data = nn_step_life.weight.data * 0 + 1 
  nn_step_life.bias.data *= 0
  pitch_new = nn_step_life(pitch)
  return (pitch == 0) * 1. * (pitch_new == 3) + (pitch == 1) * 1. * ((pitch_new == 2)  + (pitch_new == 3))

In [ ]:
def new_pitchs(cnt_life, height, width):
  pitch = torch.rand(cnt_life,1,height, width)
  density = (torch.ones(cnt_life,1,height, width).view(cnt_life,-1) * torch.rand(cnt_life).view(cnt_life,-1)).view(cnt_life,1,height, width)
  pitch = (pitch > density) * 1.
  return pitch

In [ ]:
#class BCELoss2d(torch.nn.BCELoss):
  #def __init__(self, weight):
  #  super().__init__(self, weight)
  
#  def forward(self, input, target):
#    input = input.view(-1)
#    target = target.view(-1)
#    super().forward(input, target)


In [ ]:
#(x_train == y_train) * 1 / (x_train.shape)
#x_train.shape
#torch.prod(x_train.shape)

#accuracy_score_2d(x_train, y_train)




In [ ]:
def accuracy_score_2d(input, target):
  #print(input.shape)

  #input = input.argmax(dim=1) * 1.
  input = (input > 0.5) * 1.
  target = (target > 0.5) * 1.  
  return 1-  ((input == target) * 1.).sum() / input.shape.numel()



In [ ]:
pitch = new_pitchs(CNT_LIFE, HEIGHT, WIDTH)

for step_life in range(CNT_STEP_LIFE):
  pitch_new = one_step_life(pitch)
  if step_life == 5:
    x_train = pitch_new
    y_train = pitch
  elif step_life > 5:
    x_train = torch.cat((x_train, pitch_new), dim = 0)
    y_train = torch.cat((y_train, pitch), dim = 0)
  
  pitch = pitch_new[(pitch_new.sum(dim=(1,2,3)) > 0) & ((pitch != pitch_new).sum(dim=(1,2,3)) > 0)]

del pitch, pitch_new


In [ ]:
pitch = new_pitchs(CNT_LIFE, HEIGHT, WIDTH)

for step_life in range(CNT_STEP_LIFE):
  pitch_new = one_step_life(pitch)
  if step_life == 5:
    x_val = pitch_new
    y_val = pitch
  elif step_life > 5:
    x_val = torch.cat((x_val, pitch_new), dim = 0)
    y_val = torch.cat((y_val, pitch), dim = 0)
  
  pitch = pitch_new[(pitch_new.sum(dim=(1,2,3)) > 0) & ((pitch != pitch_new).sum(dim=(1,2,3)) > 0)]

del pitch, pitch_new


In [ ]:
x_train.shape, x_val.shape

(torch.Size([118337, 1, 25, 25]), torch.Size([118766, 1, 25, 25]))

In [ ]:
#x_train.to(conf.device)
#y_train.to(conf.device)


In [ ]:
train_dataset = LifeDataset(x_train, x_train)
val_dataset = LifeDataset(x_val, x_val)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=16)

In [ ]:
conf = Configurations()
conf.device = 'cuda:0'
conf.cnt_epoch = 15
conf.lr = 3e-3

#loss = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.,10.]))
loss = torch.nn.BCELoss()
#loss = LossLife()

metrics = {'acc' :accuracy_score_2d}
optimizer = torch.optim.Adam
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau

#model.to(conf.device)

const = Сonstructor(model, loss, metrics, optimizer, scheduler, conf)


In [ ]:
const.fit(train_dataloader, val_dataloader)


train epoch 0 loss = 81796124.500 acc = 0.175 time 50.96
val epoch 0 loss = 37059862.317 acc = 0.092 time 13.21
train epoch 1 loss = 28269895.854 acc = 0.092 time 52.06
val epoch 1 loss = 24117228.333 acc = 0.092 time 13.32
train epoch 2 loss = 20569630.242 acc = 0.092 time 53.17
val epoch 2 loss = 19444942.317 acc = 0.092 time 13.48
train epoch 3 loss = 16432129.883 acc = 0.092 time 53.79
val epoch 3 loss = 14244699.517 acc = 0.092 time 13.61
train epoch 4 loss = 13484251.629 acc = 0.092 time 53.80
val epoch 4 loss = 13524754.158 acc = 0.092 time 13.55
train epoch 5 loss = 12661373.800 acc = 0.092 time 53.93
val epoch 5 loss = 12303368.717 acc = 0.092 time 13.52
train epoch 6 loss = 11963508.835 acc = 0.092 time 53.97
val epoch 6 loss = 12363894.917 acc = 0.092 time 13.57
train epoch 7 loss = 11501602.575 acc = 0.092 time 53.92
val epoch 7 loss = 11796946.642 acc = 0.092 time 13.58
train epoch 8 loss = 10990951.304 acc = 0.091 time 53.82
val epoch 8 loss = 10761552.542 acc = 0.091 tim

In [ ]:
const.fit(train_dataloader, val_dataloader)

train epoch 15 loss = 9366723.850 acc = 0.086 time 53.08
val epoch 15 loss = 9364175.238 acc = 0.087 time 13.44
Epoch  1001: reducing learning rate of group 0 to 2.7000e-04.
train epoch 16 loss = 9308424.721 acc = 0.086 time 53.57
val epoch 16 loss = 9317549.375 acc = 0.086 time 13.56
train epoch 17 loss = 9280736.121 acc = 0.086 time 53.85
val epoch 17 loss = 9308583.175 acc = 0.086 time 13.55
Epoch  1102: reducing learning rate of group 0 to 8.1000e-05.
train epoch 18 loss = 9274921.342 acc = 0.086 time 54.01
val epoch 18 loss = 9297186.804 acc = 0.086 time 13.55
train epoch 19 loss = 9262666.733 acc = 0.086 time 53.80
val epoch 19 loss = 9289038.804 acc = 0.086 time 13.58
Epoch  1241: reducing learning rate of group 0 to 2.4300e-05.
train epoch 20 loss = 9259044.717 acc = 0.086 time 54.09
val epoch 20 loss = 9286656.608 acc = 0.086 time 13.56
train epoch 21 loss = 9253114.787 acc = 0.086 time 54.03
val epoch 21 loss = 9285472.700 acc = 0.086 time 13.63
Epoch  1342: reducing learning

In [ ]:
#loss(x_train, x_train)
#loss = loss.to(conf.device)
#x_train = x_train[0:2].to(conf.device)
#x_train.device


#x_train[0:1].permute(0, 2, 3, 1).shape
#x_train[0:2].sum((1))


In [ ]:
!nvidia-smi

Mon Sep 28 01:22:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   85C    P0    32W /  75W |   7225MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------